In [1]:
from pathlib import Path
from importlib import reload

import pandas as pd
import numpy as np
from tqdm.auto import trange, tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import sys
sys.path.insert(0, '/app')
import libs.collections as lc
from torchvision.transforms import Compose, ToTensor
from torch.utils.tensorboard import SummaryWriter

from libs.metrics import score_func

In [2]:
import submission_model as sm

In [3]:
model = sm.Model()

In [5]:
data_dir = Path('/_data/data_csv/')

dataframes = {}
for file in tqdm(data_dir.iterdir()):
    name = file.stem.split('_')[1]
    train, test = train_test_split(pd.read_csv(file.as_posix(), dtype=np.float32), random_state=seed)
    dataframes[name]  =  {'train': train, 'test': test}
    
condition_cols = ['TrackP', 'TrackEta', 'NumLongTracks']
target_cols = ['RichDLLbt', 'RichDLLk', 'RichDLLmu', 'RichDLLp', 'RichDLLe']


# transformers = dict()

# for particle in tqdm(dataframes):
#     tr = QuantileTransformer(output_distribution='normal',random_state=seed)
#     dataframes[particle]['train'][:] = tr.fit_transform(
#         dataframes[particle]['train']
#     ).astype(np.float32)

#     dataframes[particle]['test'][:] = tr.transform(dataframes[particle]['test']).astype(np.float32)
#     transformers[particle] = tr
    

In [31]:
def plot_hist(x, y):
    fig = plt.figure(frameon=False);
    ax = fig.subplots(1);
    _, bins, _ = ax.hist(x,bins=100, label='true', alpha=0.7);
    _ = ax.hist(y, bins=bins, label='fake', alpha=0.7);
    fig.legend();
    plt.close(fig)
    return fig


In [33]:
import matplotlib.pyplot as plt

In [ ]:
for particle in tqdm(['proton', 'muon', 'kaon', 'pion']):
    c_train = dataframes[particle]['train'][condition_cols]
    x_train = dataframes[particle]['train'][target_cols]
    c_test = dataframes[particle]['test'][condition_cols]
    x_test = dataframes[particle]['test'][target_cols]
    model = sm.Model()
    model.fit(c_train, x_train)
    predicted = model.predict(c_test)
    reference= np.c_[x_test.values, c_test.values]
    generated = np.c_[predicted.values, c_test.values]
    score = score_func(generated, reference)
    print(particle, ': ', score)
    for col in target_cols:
        fig = plot_hist(x_test[col].values, predicted[col].values)
        display(fig)
    print()
        